In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

import os

from matplotlib import pyplot as plt

from metavision_core.event_io import RawReader

import lib

In [2]:
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['svg.fonttype'] = 'none'

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

#### Latex PGF Plot exports


In [3]:
SAVE_PGF = True

import matplotlib

if SAVE_PGF:
	#latex pgfcode export

	matplotlib.use("pgf")
	matplotlib.rcParams.update({
		"pgf.texsystem": "pdflatex",
		'font.family': 'serif',
		'text.usetex': True,
		'pgf.rcfonts': False,
	})

#### Loading the dataset

In [4]:
import numpy as np

dataset_path = "dataset/45/"

filenames = np.array(lib.load_filenames_to_matrix(dataset_path))

raws = []
for fxs in filenames:
	rawsx = [RawReader(f) for f in fxs]
	raws.append(rawsx)

distances = [1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 4.0, 5.0]
frequencies = [10, 25, 50, 100, 250, 500, 1000, 2500, 5000, 10000, 20000, 30000]
bin_width_us = 1000

#### Get the events from raws

In [5]:
events = lib.recursive_map(lib.raw_load_events, raws, dtime=1000000, start_ts=0.1 * 1e6)

#### Process the data

In [6]:
#check if exists "avg_events.npy (precomputed) - takes a long time to compute"
avg_events_array = None

if os.path.exists("data/avg_events.npz"):
	k = ['avg_events_array_45', 'std_events_array_45', 'avg_events_array_0', 'std_events_array_0']
	npz = np.load("data/avg_events.npz")
	avg_events_array_45 = npz[k[0]]
	std_events_array_45 = npz[k[1]]
	avg_events_array_0 = npz[k[2]]
	std_events_array_0 = npz[k[3]]

else:
	avg_events_matrix = []
	std_events_matrix = []

	for distance_idx, event_list in enumerate(events):
		distance_avg_events = []
		distance_std_events = []
		print(f"Processing distance {distances[distance_idx]} meters")
		for freq_idx, event_set in enumerate(event_list):
			frequency = frequencies[freq_idx]
			
			if len(event_set) == 0:
				print(f"  No events for frequency {frequency} Hz")
				distance_avg_events.append(np.nan)  #NaN for missing data
				continue
			
			#if needs to be adjusted
			prominence_value = None
			
			#event data processing
			vals_dict = lib.process_event_data(
				event_set,
				frequency,
				prominence_value=prominence_value
			)

			avg_events, std_events, event_counts, signal, time_axis, peak_indices = vals_dict.values()
			
			print(f"  Frequency: {frequency} Hz, Average Events per Period: {avg_events}")
			
			distance_avg_events.append(avg_events)
			distance_std_events.append(std_events)
			
			# plt.figure(figsize=(12, 6))
			# plt.plot(time_axis, signal)
			# plt.plot(time_axis[peak_indices], signal[peak_indices], 'rx')  # Mark peaks
			# plt.xlabel('Time (us)')
			# plt.ylabel('Summed Polarity')
			# plt.title(f'Distance {distances[distance_idx]} m, Frequency {frequency} Hz')
			# plt.show()
		
		avg_events_matrix.append(distance_avg_events)
		std_events_matrix.append(distance_std_events)

	avg_events_array = np.array(avg_events_matrix)
	std_events_array = np.array(std_events_matrix)

In [7]:
#save the events array to npz
#load the _0 from the npz file

#avg_events_array_0 = np.load("data/avg_events_0.npz")["avg_events_array_0"]
#std_events_array_0 = np.load("data/avg_events_0.npz")["std_events_array_0"]

#np.savez("data/avg_events.npz", avg_events_array_45=avg_events_array, std_events_array_45=std_events_array, avg_events_array_0=avg_events_array_0, std_events_array_0=std_events_array_0)

### Plot the effect of distance

In [8]:
evs = avg_events_array_0
stdevs = std_events_array_0

In [9]:
lib.plot_avg_events_vs_distance(distances, evs, stdevs, frequencies, save_pgf=SAVE_PGF)
lib.plot_log_avg_events_vs_distance(distances, evs, stdevs, frequencies, save_pgf=SAVE_PGF)

Saving plot to avg_events_vs_distance.pgf
Saving plot to log_avg_events_vs_distance.pgf


### Plot the effect of frequency

In [10]:
lib.plot_avg_events_vs_frequency(frequencies, evs, stdevs, distances, save_pgf=SAVE_PGF)
lib.plot_log_avg_events_vs_frequency(frequencies, evs, stdevs, distances, save_pgf=SAVE_PGF)

Saving plot to avg_events_vs_frequency.pgf
Saving plot to log_avg_events_vs_frequency.pgf


#### Fitting a polynomial curve to the data using *scipy.optimize.curve_fit*

In [11]:
avg_events_array = avg_events_array_0.copy()

In [12]:
from scipy.optimize import curve_fit

def power_law(x, k, n):
	return k * x ** (-n)

distances_array = np.array(distances)

k_fits = []
n_fits = []

#plot optimal fit for each frequency
for freq_idx in range(avg_events_array.shape[1]):
	avg_events = avg_events_array[:, freq_idx]

	valid_indices = ~np.isnan(avg_events)
	distances_valid = distances_array[valid_indices]
	avg_events_valid = avg_events[valid_indices]

	if len(distances_valid) == 0 or len(avg_events_valid) == 0:
		print(f"Skipping frequency index {freq_idx} due to lack of valid data.")
		continue

	popt, pcov = curve_fit(power_law, distances_valid, avg_events_valid)
	k_fit, n_fit = popt

	k_fits.append(k_fit)
	n_fits.append(n_fit)

	# plt.figure()
	# plt.loglog(distances_valid, avg_events_valid, 'o', label='Measured Data')
	# plt.loglog(distances_valid, power_law(distances_valid, *popt), '-', 
	# 		label=f'Fit: y = {k_fit:.2e} * x^(-{n_fit:.2f})')
	# plt.xlabel('Distance (m)')
	# plt.ylabel('Average Events per Period')
	# plt.title(f'Average Events vs. Distance (Log-Log Scale) for Frequency Index {freq_idx}')
	# plt.legend()
	# plt.grid(True, which="both", ls="--")
	# plt.show()

	print(f"Frequency index {freq_idx}: Fitted exponent n: {n_fit:.2f}")

#exponents n
plt.clf()
plt.figure()
plt.plot(frequencies, n_fits, 'o-', label='Fitted Exponent n')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Fitted Exponent n')
plt.title('Fitted Exponent n vs. Frequency')
plt.legend()
plt.grid(True)
plt.show()

#coefficients k

# plt.figure()
# plt.plot(frequencies, k_fits, 'o-', label='Fitted Coefficient k')
# plt.xlabel('Frequency (Hz)')
# plt.ylabel('Fitted Coefficient k')
# plt.title('Fitted Coefficient k vs. Frequency')
# plt.legend()
# plt.grid(True)
# plt.show()

Frequency index 0: Fitted exponent n: 2.06
Frequency index 1: Fitted exponent n: 1.64
Frequency index 2: Fitted exponent n: 1.54
Frequency index 3: Fitted exponent n: 1.51
Frequency index 4: Fitted exponent n: 1.47
Frequency index 5: Fitted exponent n: 1.38
Frequency index 6: Fitted exponent n: 1.30
Frequency index 7: Fitted exponent n: 1.28
Frequency index 8: Fitted exponent n: 1.31
Frequency index 9: Fitted exponent n: 1.40
Frequency index 10: Fitted exponent n: 0.50
Frequency index 11: Fitted exponent n: -0.95


/tmp/ipykernel_78489/800505630.py:51: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


#### Fitting $x^{-2}$, $x^{-3}$, $x^{-4}$ and $y = c + ax^{-2} + bx^{-3} + dx^{-4} + fe^x$


In [13]:
# this is the frequency which will be used for the fitting plot

freq_idx = 5

In [14]:
def power_law(x, k, n):
	return k * x ** (-n)

distances_array = np.array(distances)
avg_events = avg_events_array[:, freq_idx]

valid_indices = ~np.isnan(avg_events)
distances_valid = distances_array[valid_indices]
avg_events_valid = avg_events[valid_indices]

#save the xy values of the avg events 
#to .csv in docs/fig/plots/inverse_square/data/data.csv
#np.savetxt("docs/fig/plots/inverse_square/data/data.csv", np.column_stack([distances_valid, avg_events_valid]), delimiter=",", header="Distance (m),Average Events per Period")

X_custom = np.column_stack([
	np.ones_like(distances_valid),
	distances_valid**-2,
	distances_valid**-3,
	distances_valid**-4,
	np.exp(distances_valid),
])

y_custom = avg_events_valid


coefficients_custom, residuals_custom, rank_custom, s_custom = np.linalg.lstsq(X_custom, y_custom, rcond=None)
c_fit, b_fit, d_fit, f_fit, e_fit = coefficients_custom
y_fit_custom = X_custom @ coefficients_custom

ss_res_custom = np.sum((y_custom - y_fit_custom) ** 2)
ss_tot_custom = np.sum((y_custom - np.mean(y_custom)) ** 2)
r2_custom = 1 - (ss_res_custom / ss_tot_custom)

fits = []
labels = []
r2_scores = []

# Fit x**-2
X_2 = np.column_stack([np.ones_like(distances_valid), distances_valid**-2])
coefficients_2, residuals_2, rank_2, s_2 = np.linalg.lstsq(X_2, y_custom, rcond=None)
k_fit_2, n_fit_2 = coefficients_2
y_fit_2 = X_2 @ coefficients_2
fits.append(y_fit_2)
labels.append(f'Fit: y = {k_fit_2:.2e} * x^(-2)')

# R**2 for x**-2
ss_res_2 = np.sum((y_custom - y_fit_2) ** 2)
r2_2 = 1 - (ss_res_2 / ss_tot_custom)
r2_scores.append(r2_2)

# Fit x**-3
X_3 = np.column_stack([np.ones_like(distances_valid), distances_valid**-3])
coefficients_3, residuals_3, rank_3, s_3 = np.linalg.lstsq(X_3, y_custom, rcond=None)
k_fit_3, n_fit_3 = coefficients_3
y_fit_3 = X_3 @ coefficients_3
fits.append(y_fit_3)
labels.append(f'Fit: y = {k_fit_3:.2e} * x^(-3)')

# R**2 for x**-3
ss_res_3 = np.sum((y_custom - y_fit_3) ** 2)
r2_3 = 1 - (ss_res_3 / ss_tot_custom)
r2_scores.append(r2_3)

# Fit x**-4
X_4 = np.column_stack([np.ones_like(distances_valid), distances_valid**-4])
coefficients_4, residuals_4, rank_4, s_4 = np.linalg.lstsq(X_4, y_custom, rcond=None)
k_fit_4, n_fit_4 = coefficients_4
y_fit_4 = X_4 @ coefficients_4
fits.append(y_fit_4)
labels.append(f'Fit: y = {k_fit_4:.2e} * x^(-4)')

# R**2 for x**-4
ss_res_4 = np.sum((y_custom - y_fit_4) ** 2)
r2_4 = 1 - (ss_res_4 / ss_tot_custom)
r2_scores.append(r2_4)

plt.clf()
plt.figure(figsize=(12, 6))
plt.plot(distances_valid, avg_events_valid, 'o', label='Measured Data')
plt.plot(distances_valid, y_fit_custom, '-', label=f'Custom Fit: y = {c_fit:.2e} + {b_fit:.2e} * x^-2 + {d_fit:.2e} * x^-3 + {f_fit:.2e} * x^-4 + {e_fit:.2e} * e^x')

for fit, label in zip(fits, labels):
	plt.plot(distances_valid, fit, '-', label=label)

plt.xlabel('Distance (m)')
plt.ylabel('Average Events per Period')
plt.title('Average Events vs. Distance (Combined Fits)')
plt.legend()
plt.grid(True)

if SAVE_PGF:
	plt.savefig("combined_fit.pgf")
else:
	plt.show()

print(f"Custom Fit coefficients:\n c = {c_fit:.2e}, b = {b_fit:.2e}, d = {d_fit:.2e}, f = {f_fit:.2e}, e = {e_fit:.2e}")
print(f"Custom Fit R^2: {r2_custom:.2f}")

for i, (label, r2) in enumerate(zip(labels, r2_scores)):
	print(f"{label} R^2: {r2:.2f}")

Custom Fit coefficients:
 c = 8.98e+01, b = 1.33e+03, d = -4.77e+02, f = 2.95e+01, e = -1.62e-01
Custom Fit R^2: 1.00
Fit: y = 1.25e+02 * x^(-2) R^2: 0.99
Fit: y = 2.17e+02 * x^(-3) R^2: 0.94
Fit: y = 2.63e+02 * x^(-4) R^2: 0.87


In [15]:
max_distances = []

# this needs some reajdustment
detection_threshold = 10

for freq_idx, frequency in enumerate(frequencies):
	avg_events_per_distance = avg_events_array[:, freq_idx]
	valid_indices = np.where(avg_events_per_distance > detection_threshold)[0]

	if valid_indices.size > 0:
		max_distance = distances[valid_indices[-1]]
		max_distances.append(max_distance)
		
	else:
		max_distances.append(np.nan)

plt.clf()
plt.figure()
plt.plot(frequencies, max_distances, 'o-')
plt.xscale('log')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Maximum Detection Distance (m)')
plt.title('Frequency vs. Maximum Detection Distance')
plt.grid(True, which="both", ls="--")
plt.show()

/tmp/ipykernel_78489/2955461890.py:25: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


#### Investigate the LED radiation characteristics

In [16]:
# Define dataset paths
angle_data_50 = "angle_dataset/uhel/50/"
angle_data_100 = "angle_dataset/uhel/100/"
angle_data_200 = "angle_dataset/uhel/200/"

# Load filenames into matrices
angle_filenames_50 = np.array(lib.load_filenames_to_matrix(angle_data_50))
angle_filenames_100 = np.array(lib.load_filenames_to_matrix(angle_data_100))
angle_filenames_200 = np.array(lib.load_filenames_to_matrix(angle_data_200))

angle_freqs = [10, 100, 1000, 10000]
angle_distances = [0.5, 1, 2]
angles = [0, 45, 90, 135, 180]

angle_raws_50 = []
angle_raws_100 = []
angle_raws_200 = []

for fxs in angle_filenames_50:
	angle_rawsx = [RawReader(f) for f in fxs]
	angle_raws_50.append(angle_rawsx)

for fxs in angle_filenames_100:
	angle_rawsx = [RawReader(f) for f in fxs]
	angle_raws_100.append(angle_rawsx)

for fxs in angle_filenames_200:
	angle_rawsx = [RawReader(f) for f in fxs]
	angle_raws_200.append(angle_rawsx)

angle_events_50 = lib.recursive_map(lib.raw_load_events, angle_raws_50, dtime=1000000, start_ts=0.1 * 1e6)
angle_events_100 = lib.recursive_map(lib.raw_load_events, angle_raws_100, dtime=1000000, start_ts=0.1 * 1e6)
angle_events_200 = lib.recursive_map(lib.raw_load_events, angle_raws_200, dtime=1000000, start_ts=0.1 * 1e6)

def process_angle_events(angle_events, angles, angle_freqs):
	angle_avg_events_matrix = []
	angle_std_events_matrix = []

	for angle_idx, event_list in enumerate(angle_events):
		angle_avg_events = []
		angle_std_events = []
		print(f"Processing angle {angles[angle_idx]} degrees")
		for freq_idx, event_set in enumerate(event_list):
			frequency = angle_freqs[freq_idx]

			if len(event_set) == 0:
				print(f"  No events for frequency {frequency} Hz")
				angle_avg_events.append(np.nan)  # NaN for missing data
				angle_std_events.append(np.nan)  # NaN for missing data
				continue

			# if needs to be adjusted
			prominence_value = None

			vals_dict = lib.process_event_data(
				event_set,
				frequency,
				prominence_value=prominence_value
			)

			avg_events, std_events, event_counts, signal, time_axis, peak_indices = vals_dict.values()

			print(f"  Frequency: {frequency} Hz, Average Events per Period: {avg_events}")

			angle_avg_events.append(avg_events)
			angle_std_events.append(std_events)

		angle_avg_events_matrix.append(angle_avg_events)
		angle_std_events_matrix.append(angle_std_events)

	return np.array(angle_avg_events_matrix, dtype=np.float64), np.array(angle_std_events_matrix, dtype=np.float64)

#check if exists "data/angle_avg_events.npz" to save execution time

if not os.path.exists("data/angle_avg_events.npz"):
	angle_avg_events_array_50, angle_std_events_array_50 = process_angle_events(angle_events_50, angles, angle_freqs)
	angle_avg_events_array_100, angle_std_events_array_100 = process_angle_events(angle_events_100, angles, angle_freqs)
	angle_avg_events_array_200, angle_std_events_array_200 = process_angle_events(angle_events_200, angles, angle_freqs)
	np.savez("data/angle_avg_events.npz",
			angle_avg_events_array_50=angle_avg_events_array_50,
			angle_std_events_array_50=angle_std_events_array_50,
			angle_avg_events_array_100=angle_avg_events_array_100,
			angle_std_events_array_100=angle_std_events_array_100,
			angle_avg_events_array_200=angle_avg_events_array_200,
			angle_std_events_array_200=angle_std_events_array_200)
else:
	npz = np.load("data/angle_avg_events.npz", allow_pickle=True)
	angle_avg_events_array_50 = npz["angle_avg_events_array_50"]
	angle_std_events_array_50 = npz["angle_std_events_array_50"]
	angle_avg_events_array_100 = npz["angle_avg_events_array_100"]
	angle_std_events_array_100 = npz["angle_std_events_array_100"]
	angle_avg_events_array_200 = npz["angle_avg_events_array_200"]
	angle_std_events_array_200 = npz["angle_std_events_array_200"]

print(angle_std_events_array_50)
print(angle_avg_events_array_200)

plt.rcParams['figure.figsize'] = [6, 4]

lib.plot_log_avg_events_vs_angle(angles, angle_avg_events_array_50, angle_std_events_array_50, angle_freqs, label=" at .5 m", save_pgf=SAVE_PGF)
lib.plot_log_avg_events_vs_angle(angles, angle_avg_events_array_100, angle_std_events_array_100, angle_freqs, label=" at 1 m", save_pgf=SAVE_PGF)
lib.plot_log_avg_events_vs_angle(angles, angle_avg_events_array_200, angle_std_events_array_200, angle_freqs, label=" at 2 m", save_pgf=SAVE_PGF)

plt.rcParams['figure.figsize'] = [8, 6]

[[1.86845367e+04 9.46799478e+02 1.84750667e+01 1.88536365e+01]
 [0.00000000e+00 2.27729391e+03 5.17350448e+01 3.89352614e+01]
 [3.54655000e+04 1.64136329e+02 2.90448300e+01 3.13664507e+01]
 [6.57000000e+02 2.02327726e+03 2.33773497e+01 2.01807700e+01]
 [2.61980067e+02 1.52117039e+03 1.30831108e+02 3.66280095e+01]]
[[1.90552000e+04 2.86706962e+03 5.31493359e+02 3.75408422e+01]
 [3.14978000e+04 4.18465605e+03 8.24168178e+02 7.70829479e+01]
 [4.08230625e+04 5.02296970e+03 9.03364376e+02 8.71847589e+01]
 [3.38358824e+04 4.28525000e+03 7.93555556e+02 6.40969231e+01]
 [2.34617333e+04 3.22028736e+03 6.09887430e+02 5.26114365e+01]]
Saving plot to log_avg_events_vs_angle__05_m.pgf
Saving plot to log_avg_events_vs_angle__1_m.pgf
Saving plot to log_avg_events_vs_angle__2_m.pgf


### Modelling the LED intensity distribution

In [17]:
def gaussian_intensity(angle, direction, sigma=30):
	angle_diff = (angle - direction + 180) % 360 - 180
	return np.exp(-0.5 * (angle_diff / sigma) ** 2)

##########
#configurable parameters
led_sigma = 35
illumination_angle = 180
led1_position = np.array([0, 0])
led2_position = np.array([0, 0])

led1_direction = 45
led2_direction = -45

led1_intensity_factor = 1.0  # Increase this value to make LED 1 brighter
led2_intensity_factor = 1.0  # Increase this value to make LED 2 brighter

#XY grid
x = np.linspace(-0.5, 1.0, 400)
y = np.linspace(-1, 1, 400)
x, y = np.meshgrid(x, y)
##########

angles_to_led1 = np.degrees(np.arctan2(y - led1_position[1], x - led1_position[0]))
angles_to_led2 = np.degrees(np.arctan2(y - led2_position[1], x - led2_position[0]))
angles_to_led1 = (angles_to_led1 + 360) % 360
angles_to_led2 = (angles_to_led2 + 360) % 360

intensity1 = led1_intensity_factor * gaussian_intensity(angles_to_led1, led1_direction, sigma=led_sigma)
intensity2 = led2_intensity_factor * gaussian_intensity(angles_to_led2, led2_direction, sigma=led_sigma)

intensity1[(angles_to_led1 > illumination_angle) & (angles_to_led1 < (360 - illumination_angle))] = 0
intensity2[(angles_to_led2 > illumination_angle) & (angles_to_led2 < (360 - illumination_angle))] = 0

distance_to_led1 = np.sqrt((x - led1_position[0])**2 + (y - led1_position[1])**2)
distance_to_led2 = np.sqrt((x - led2_position[0])**2 + (y - led2_position[1])**2)
epsilon = 1e-6
distance_to_led1 = np.maximum(distance_to_led1, epsilon)
distance_to_led2 = np.maximum(distance_to_led2, epsilon)

intensity1 = intensity1 / (distance_to_led1**2 + 0.5)
intensity2 = intensity2 / (distance_to_led2**2 + 0.5)

combined_intensity = intensity1 + intensity2
combined_intensity = combined_intensity / np.max(combined_intensity)

plt.clf()
plt.figure(figsize=(12, 6))
contour = plt.contourf(x, y, combined_intensity, levels=50, cmap='viridis')
plt.colorbar(contour)
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Combined Intensity Contours of Two LEDs with Increased Intensity')
plt.grid(True)
plt.show()

/tmp/ipykernel_78489/3545130026.py:55: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


#### Interactive plot

In [18]:
try:
	from ipywidgets import interact, FloatSlider, IntSlider

	@interact(
		led_sigma=(10, 100, 1),
		illumination_angle=(0, 360, 1),
		led1_x=(-1.0, 1.0, 0.1),
		led1_y=(-1.0, 1.0, 0.1),
		led2_x=(-1.0, 1.0, 0.1),
		led2_y=(-1.0, 1.0, 0.1),
		led1_direction=(-180, 180, 1),
		led2_direction=(-180, 180, 1),
		led1_intensity_factor=(0.1, 5.0, 0.1),
		led2_intensity_factor=(0.1, 5.0, 0.1),
		x_line=(0.1, 1.5, 0.01)
	)
	def plot_leds(
		led_sigma=35,
		illumination_angle=180,
		led1_x=0.0,
		led1_y=0.0,
		led2_x=0.0,
		led2_y=0.0,
		led1_direction=45,
		led2_direction=-45,
		led1_intensity_factor=1.0,
		led2_intensity_factor=1.0,
		x_line=0.2
	):
		def gaussian_intensity(angle, direction, sigma):
			angle_diff = (angle - direction + 180) % 360 - 180
			return np.exp(-0.5 * (angle_diff / sigma) ** 2)
		
		def lamberian_intensity(angle, direction):
			angle_diff = (angle - direction + 180) % 360 - 180
			return np.maximum(np.cos(np.radians(angle_diff)), 0)

		led1_position = np.array([led1_x, led1_y])
		led2_position = np.array([led2_x, led2_y])

		x_vals = np.linspace(-0.5, 1.5, 400)
		y_vals = np.linspace(-1, 1, 400)
		x, y = np.meshgrid(x_vals, y_vals)

		angles_to_led1 = np.degrees(np.arctan2(y - led1_position[1], x - led1_position[0]))
		angles_to_led2 = np.degrees(np.arctan2(y - led2_position[1], x - led2_position[0]))
		angles_to_led1 = (angles_to_led1 + 360) % 360
		angles_to_led2 = (angles_to_led2 + 360) % 360

		#intensity1 = led1_intensity_factor * gaussian_intensity(angles_to_led1, led1_direction, sigma=led_sigma)
		#intensity2 = led2_intensity_factor * gaussian_intensity(angles_to_led2, led2_direction, sigma=led_sigma)

		intensity1 = led1_intensity_factor * lamberian_intensity(angles_to_led1, led1_direction)
		intensity2 = led2_intensity_factor * lamberian_intensity(angles_to_led2, led2_direction)

		intensity1[(angles_to_led1 > illumination_angle) & (angles_to_led1 < (360 - illumination_angle))] = 0
		intensity2[(angles_to_led2 > illumination_angle) & (angles_to_led2 < (360 - illumination_angle))] = 0

		distance_to_led1 = np.hypot(x - led1_position[0], y - led1_position[1]) + 1e-6
		distance_to_led2 = np.hypot(x - led2_position[0], y - led2_position[1]) + 1e-6

		intensity1 /= (distance_to_led1**2 + 0.5)
		intensity2 /= (distance_to_led2**2 + 0.5)

		combined_intensity = intensity1 + intensity2
		combined_intensity /= np.max(combined_intensity)

		plt.figure(figsize=(12, 10))

		plt.subplot(2, 1, 1)
		contour = plt.contourf(x, y, combined_intensity, levels=50, cmap='inferno')
		plt.colorbar(contour, label='Normalized Intensity')
		plt.xlabel('X Position')
		plt.ylabel('Y Position')
		plt.title('Combined Intensity Contours')
		plt.grid(True)

		plt.axvline(x=x_line, color='grey', linestyle='--', linewidth=2)

		x_index = np.abs(x_vals - x_line).argmin()
		y_line = y_vals
		intensity_line = combined_intensity[:, x_index]

		plt.subplot(2, 1, 2)
		plt.plot(y_line, intensity_line)
		plt.xlabel('Y Position')
		plt.ylabel('Intensity')
		plt.title(f'Intensity at X = {x_line:.2f}')
		plt.grid(True)

		plt.tight_layout()
		plt.show()

except ImportError:
	print('Please review installed packages.')

interactive(children=(IntSlider(value=35, description='led_sigma', min=10), IntSlider(value=180, description='…

In [19]:
@interact(
    led_sigma=(10, 100, 1),
    illumination_angle=(0, 360, 1),
    led1_x=(-1.0, 1.0, 0.1),
    led1_y=(-1.0, 1.0, 0.1),
    led1_direction=(-180, 180, 1),
    led2_x=(-1.0, 1.0, 0.1),
    led2_y=(-1.0, 1.0, 0.1),
    led2_direction=(-180, 180, 1),
    led1_intensity_factor=(0.1, 5.0, 0.1),
    led2_intensity_factor=(0.1, 5.0, 0.1)
)
def plot_leds_polar(
    led_sigma=35,
    illumination_angle=180,
    led1_x=0.0,
    led1_y=0.0,
    led1_direction=45,
    led2_x=0.0,
    led2_y=0.0,
    led2_direction=-45,
    led1_intensity_factor=1.0,
    led2_intensity_factor=1.0
):
    def gaussian_intensity(angle, direction, sigma):
        angle_diff = (angle - direction + 180) % 360 - 180
        return np.exp(-0.5 * (angle_diff / sigma) ** 2)

    # LED positions
    led1_position = np.array([led1_x, led1_y])
    led2_position = np.array([led2_x, led2_y])
    
    # Create grid
    x_vals = np.linspace(-1.5, 1.5, 400)
    y_vals = np.linspace(-1.5, 1.5, 400)
    x, y = np.meshgrid(x_vals, y_vals)

    # Compute polar angles
    angles_to_led1 = np.degrees(np.arctan2(y - led1_position[1], x - led1_position[0]))
    angles_to_led2 = np.degrees(np.arctan2(y - led2_position[1], x - led2_position[0]))
    angles_to_led1 = (angles_to_led1 + 360) % 360
    angles_to_led2 = (angles_to_led2 + 360) % 360

    # Compute Gaussian intensities
    intensity1 = led1_intensity_factor * gaussian_intensity(angles_to_led1, led1_direction, led_sigma)
    intensity2 = led2_intensity_factor * gaussian_intensity(angles_to_led2, led2_direction, led_sigma)
    
    # Set illumination limits
    intensity1[(angles_to_led1 > illumination_angle) & (angles_to_led1 < (360 - illumination_angle))] = 0
    intensity2[(angles_to_led2 > illumination_angle) & (angles_to_led2 < (360 - illumination_angle))] = 0

    # Combine intensities
    combined_intensity = intensity1 + intensity2
    combined_intensity /= np.max(combined_intensity)

    # Aggregate data for polar plot
    angles = np.linspace(0, 360, 360)
    intensity1_polar = gaussian_intensity(angles, led1_direction, led_sigma)
    intensity2_polar = gaussian_intensity(angles, led2_direction, led_sigma)
    combined_polar = intensity1_polar + intensity2_polar

    # Plot results
    plt.clf()
    plt.figure(figsize=(12, 10))

    # Intensity contours
    plt.subplot(2, 1, 1)
    plt.contourf(x, y, combined_intensity, levels=50, cmap='inferno')
    plt.colorbar(label='Normalized Intensity')
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.title('Combined Intensity Contours')
    plt.grid(True)

    # Polar plot
    plt.subplot(2, 1, 2, polar=True)
    plt.plot(np.radians(angles), combined_polar, label='Combined Intensity')
    plt.plot(np.radians(angles), intensity1_polar, label='LED1 Intensity', linestyle='--')
    plt.plot(np.radians(angles), intensity2_polar, label='LED2 Intensity', linestyle='--')
    plt.legend(loc='upper right')
    plt.title('Intensity Distribution by Angle')
    
    plt.tight_layout()
    plt.show()

interactive(children=(IntSlider(value=35, description='led_sigma', min=10), IntSlider(value=180, description='…